In [7]:
import json
import os,sys

In [8]:
label_file_path = '/home/kimseungjun/datasets/My_PT_data/PT_data/train/label.json'
with open(label_file_path,'r') as f:
    labels = json.load(f)

#print(labels)
new_label = {}
for path_name,cls_la in labels.items():
    bn = os.path.basename(path_name)
    new_label[bn]=cls_la


In [ ]:
print(new_label)
new_label_file_path = '/home/kimseungjun/datasets/My_PT_data/PT_data/train/bn_label.json'

with open(new_label_file_path, 'w') as f:
    json.dump(new_label, f, indent=4)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def find_points(obj):
    """
    obj 안에서 point cloud로 보이는 Nx3 배열/텐서를 찾아 반환.
    가장 흔한 키/형태를 우선 탐색.
    """
    # 텐서/넘파이로 바로 들어있는 경우
    if isinstance(obj, torch.Tensor):
        arr = obj.detach().cpu().numpy()
        if arr.ndim == 2 and arr.shape[1] >= 3:
            return arr[:, :3]
    if isinstance(obj, np.ndarray):
        if obj.ndim == 2 and obj.shape[1] >= 3:
            return obj[:, :3]

    # dict 형태
    if isinstance(obj, dict):
        candidate_keys = ["points", "xyz", "pos", "coords", "coord", "pcd", "pointcloud", "verts", "vertices"]
        for k in candidate_keys:
            if k in obj:
                pts = find_points(obj[k])
                if pts is not None:
                    return pts
        # dict 값들 전체를 훑어보기
        for v in obj.values():
            pts = find_points(v)
            if pts is not None:
                return pts

    # list/tuple 형태
    if isinstance(obj, (list, tuple)):
        for v in obj:
            pts = find_points(v)
            if pts is not None:
                return pts

    return None

def vis_pth_pointcloud(pth_path, max_points=200_000, elev=20, azim=45):
    data = torch.load(pth_path, map_location="cpu")
    pts = find_points(data)
    if pts is None:
        raise ValueError("이 .pth에서 Nx3 points를 찾지 못했어. (키 이름/구조가 특이한 듯)")

    # 다운샘플
    n = pts.shape[0]
    if n > max_points:
        idx = np.random.choice(n, max_points, replace=False)
        pts = pts[idx]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], s=0.5)
    ax.view_init(elev=elev, azim=azim)
    ax.set_title(f"{pth_path}  (N={pts.shape[0]})")
    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    plt.show()

# 사용 예시
vis_pth_pointcloud("/home/kimseungjun/datasets/My_PT_data/PT_data/train/0000012632_20250918_152808.pth")

In [65]:
init_price = 10000.0

total_cost = init_price      # 총 투입금
total_qty  = 1.0             # 초기 1개 매수했다고 가정
avg_price  = total_cost / total_qty

push_money = 10000.0
buy_price = avg_price
for i in range(1, 100):
    print(i, "buy_price:", buy_price, "avg_price:", avg_price, "ratio:", avg_price/init_price, 'total_cost:',total_cost)
    buy_price = avg_price * 0.9              # 평단의 10% 아래 가격에서 매수
    add_qty   = push_money / buy_price       # 1만원으로 살 수 있는 수량

    total_cost += push_money
    total_qty  += add_qty
    avg_price   = total_cost / total_qty     # ✅ 이게 평단가

    #print(i, "buy_price:", buy_price, "avg_price:", avg_price, "ratio:", avg_price/init_price)

1 buy_price: 10000.0 avg_price: 10000.0 ratio: 1.0 total_cost: 10000.0
2 buy_price: 9000.0 avg_price: 9473.684210526315 ratio: 0.9473684210526315 total_cost: 20000.0
3 buy_price: 8526.315789473683 avg_price: 9135.33834586466 ratio: 0.9135338345864661 total_cost: 30000.0
4 buy_price: 8221.804511278195 avg_price: 8888.43730948994 ratio: 0.888843730948994 total_cost: 40000.0
5 buy_price: 7999.5935785409465 avg_price: 8695.21041145755 ratio: 0.869521041145755 total_cost: 50000.0
6 buy_price: 7825.689370311796 avg_price: 8537.115676703777 ratio: 0.8537115676703777 total_cost: 60000.0
7 buy_price: 7683.404109033399 avg_price: 8403.723244255281 ratio: 0.8403723244255281 total_cost: 70000.0
8 buy_price: 7563.350919829753 avg_price: 8288.603747758634 ratio: 0.8288603747758634 total_cost: 80000.0
9 buy_price: 7459.743372982771 avg_price: 8187.523214249381 ratio: 0.8187523214249381 total_cost: 90000.0
10 buy_price: 7368.770892824444 avg_price: 8097.550431675212 ratio: 0.8097550431675212 total_cos

In [67]:
init_price = 10000.0

total_cost = init_price
total_qty  = 1.0
avg_price  = total_cost / total_qty

push_money = 10000.0
prev_ratio = avg_price / init_price

for i in range(1, 50):
    print(i, "avg_price:", avg_price, "ratio:", avg_price/init_price, "total_cost:", total_cost)

    # 1) 기본 매수: 평단의 10% 아래 가격에서 1만원
    avg_old = avg_price
    P = avg_old * 0.9                      # buy_price
    add_qty = push_money / P

    total_cost += push_money
    total_qty  += add_qty
    avg_price   = total_cost / total_qty

    ratio = avg_price / init_price
    drop = prev_ratio - ratio

    # 2) 감소폭이 0.01보다 작으면, 추가 투입으로 목표 ratio = prev_ratio - 0.01 맞추기
    if drop < 0.01:
        target_ratio = prev_ratio - 0.01
        A = target_ratio * init_price      # 목표 평단

        C = total_cost
        Q = total_qty

        denom = (1.0 - A / P)
        # denom이 0에 매우 가까우면(수치폭발) 안전하게 스킵/클립
        if abs(denom) > 1e-12:
            x = (A * Q - C) / denom        # 추가로 넣어야 할 현금
            if x > 0:                      # 음수면 추가투입 불필요/불가능 케이스
                total_cost += x
                total_qty  += x / P
                avg_price   = total_cost / total_qty
                ratio = avg_price / init_price

                print(f"  부족({drop:.6f}) → 추가투입 x={x:.2f}원, new_ratio={ratio:.6f}")

    prev_ratio = ratio

1 avg_price: 10000.0 ratio: 1.0 total_cost: 10000.0
2 avg_price: 9473.684210526315 ratio: 0.9473684210526315 total_cost: 20000.0
3 avg_price: 9135.33834586466 ratio: 0.9135338345864661 total_cost: 30000.0
4 avg_price: 8888.43730948994 ratio: 0.888843730948994 total_cost: 40000.0
5 avg_price: 8695.21041145755 ratio: 0.869521041145755 total_cost: 50000.0
6 avg_price: 8537.115676703777 ratio: 0.8537115676703777 total_cost: 60000.0
7 avg_price: 8403.723244255281 ratio: 0.8403723244255281 total_cost: 70000.0
8 avg_price: 8288.603747758634 ratio: 0.8288603747758634 total_cost: 80000.0
9 avg_price: 8187.523214249381 ratio: 0.8187523214249381 total_cost: 90000.0
  부족(0.008997) → 추가투입 x=1269.53원, new_ratio=0.808752
10 avg_price: 8087.523214249381 ratio: 0.8087523214249381 total_cost: 101269.52881897007
  부족(0.007996) → 추가투입 x=2859.58원, new_ratio=0.798752
11 avg_price: 7987.523214249381 ratio: 0.7987523214249381 total_cost: 114129.10940613938
  부족(0.007086) → 추가투입 x=4699.94원, new_ratio=0.788752


In [71]:
init_price = 10000.0

total_cost = init_price
total_qty  = 1.0
avg_price  = total_cost / total_qty

push_money = 10000.0
prev_ratio = avg_price / init_price

for i in range(1, 1000):
    print(i, "avg_price:", avg_price, "ratio:", avg_price/init_price, "total_cost:", total_cost)

    # 1) 기본 매수: 평단의 10% 아래 가격에서 1만원
    avg_old = avg_price
    P = avg_old * 0.9                      # buy_price
    add_qty = push_money / P

    total_cost += push_money
    total_qty  += add_qty
    avg_price   = total_cost / total_qty

    ratio = avg_price / init_price
    drop = prev_ratio - ratio

    # 2) 감소폭이 0.01보다 작으면, 추가 투입으로 목표 ratio = prev_ratio - 0.01 맞추기
    # if drop < 0.01:
    #     target_ratio = prev_ratio - 0.01
    #     A = target_ratio * init_price      # 목표 평단

    #     C = total_cost
    #     Q = total_qty

    #     denom = (1.0 - A / P)
    #     # denom이 0에 매우 가까우면(수치폭발) 안전하게 스킵/클립
    #     if abs(denom) > 1e-12:
    #         x = (A * Q - C) / denom        # 추가로 넣어야 할 현금
    #         if x > 0:                      # 음수면 추가투입 불필요/불가능 케이스
    #             total_cost += x
    #             total_qty  += x / P
    #             avg_price   = total_cost / total_qty
    #             ratio = avg_price / init_price

    #             print(f"  부족({drop:.6f}) → 추가투입 x={x:.2f}원, new_ratio={ratio:.6f}")

    prev_ratio = ratio

1 avg_price: 10000.0 ratio: 1.0 total_cost: 10000.0
2 avg_price: 9473.684210526315 ratio: 0.9473684210526315 total_cost: 20000.0
3 avg_price: 9135.33834586466 ratio: 0.9135338345864661 total_cost: 30000.0
4 avg_price: 8888.43730948994 ratio: 0.888843730948994 total_cost: 40000.0
5 avg_price: 8695.21041145755 ratio: 0.869521041145755 total_cost: 50000.0
6 avg_price: 8537.115676703777 ratio: 0.8537115676703777 total_cost: 60000.0
7 avg_price: 8403.723244255281 ratio: 0.8403723244255281 total_cost: 70000.0
8 avg_price: 8288.603747758634 ratio: 0.8288603747758634 total_cost: 80000.0
9 avg_price: 8187.523214249381 ratio: 0.8187523214249381 total_cost: 90000.0
10 avg_price: 8097.550431675212 ratio: 0.8097550431675212 total_cost: 100000.0
11 avg_price: 8016.574927358461 ratio: 0.8016574927358461 total_cost: 110000.0
12 avg_price: 7943.028368391869 ratio: 0.7943028368391869 total_cost: 120000.0
13 avg_price: 7875.714568659735 ratio: 0.7875714568659735 total_cost: 130000.0
14 avg_price: 7813.70